<a href="https://colab.research.google.com/github/muhammadghazyy/pycaret/blob/main/house_price_pycaret.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pycaret

In [10]:
import pandas as pd
import numpy as np
import pycaret.regression

from google.colab import files

In [7]:
np.core.arrayprint._line_width = 80

In [2]:
files.upload()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c house-prices-advanced-regression-techniques

Saving kaggle.json to kaggle.json
  0% 0.00/31.2k [00:00<?, ?B/s]
100% 31.2k/31.2k [00:00<00:00, 55.7MB/s]
  0% 0.00/450k [00:00<?, ?B/s]
100% 450k/450k [00:00<00:00, 29.9MB/s]
  0% 0.00/13.1k [00:00<?, ?B/s]
100% 13.1k/13.1k [00:00<00:00, 12.8MB/s]
  0% 0.00/441k [00:00<?, ?B/s]
100% 441k/441k [00:00<00:00, 62.3MB/s]


In [53]:
train = pd.read_csv("train.csv")
test = pd.read_csv('test.csv')

In [54]:
testid = test['Id']

0       1461
1       1462
2       1463
3       1464
4       1465
        ... 
1454    2915
1455    2916
1456    2917
1457    2918
1458    2919
Name: Id, Length: 1459, dtype: int64

In [38]:
print(f"Jumlah baris ada sebanyak {train['Id'].count()}")

Jumlah baris ada sebanyak 1460


##Cleaning

In [22]:
nuller = pd.DataFrame({'col':train.isna().sum()})
nuller.loc[nuller['col']!=0]

,col
LotFrontage,259
Alley,1369
MasVnrType,8
MasVnrArea,8
BsmtQual,37
BsmtCond,37
BsmtExposure,38
BsmtFinType1,37
BsmtFinType2,38
Electrical,1


In [43]:
# Removing PoolQc,Fence,MiscFeature,Alley,FireplaceQu
train1 = train.copy()
train1.drop(['Id','PoolQC','Fence','MiscFeature','FireplaceQu','Alley'],inplace=True,axis=1)

In [58]:
train1

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,62.0,7917,Pave,Reg,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,5,1999,2000,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,Unf,0,Unf,0,953,953,GasA,Ex,Y,SBrkr,953,694,0,1647,0,0,2,1,3,1,TA,7,Typ,1,Attchd,1999.0,RFn,2,460,TA,TA,Y,0,40,0,0,0,0,0,8,2007,WD,Normal,175000
1456,20,RL,85.0,13175,Pave,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,Norm,1Fam,1Story,6,6,1978,1988,Gable,CompShg,Plywood,Plywood,Stone,119.0,TA,TA,CBlock,Gd,TA,No,ALQ,790,Rec,163,589,1542,GasA,TA,Y,SBrkr,2073,0,0,2073,1,0,2,0,3,1,TA,7,Min1,2,Attchd,1978.0,Unf,2,500,TA,TA,Y,349,0,0,0,0,0,0,2,2010,WD,Normal,210000
1457,70,RL,66.0,9042,Pave,Reg,Lvl,AllPub,Inside,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,9,1941,2006,Gable,CompShg,CemntBd,CmentBd,None,0.0,Ex,Gd,Stone,TA,Gd,No,GLQ,275,Unf,0,877,1152,GasA,Ex,Y,SBrkr,1188,1152,0,2340,0,0,2,0,4,1,Gd,9,Typ,2,Attchd,1941.0,RFn,1,252,TA,TA,Y,0,60,0,0,0,0,2500,5,2010,WD,Normal,266500
1458,20,RL,68.0,9717,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,5,6,1950,1996,Hip,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,TA,TA,Mn,GLQ,49,Rec,1029,0,1078,GasA,Gd,Y,FuseA,1078,0,0,1078,1,0,1,0,2,1,Gd,5,Typ,0,Attchd,1950.0,Unf,1,240,TA,TA,Y,366,0,112,0,0,0,0,4,2010,WD,Normal,142125


In [46]:
nuller2 = pd.DataFrame({'col':train1.isna().sum()})
nuller2.loc[nuller2['col']!=0]

,col
LotFrontage,259
MasVnrType,8
MasVnrArea,8
BsmtQual,37
BsmtCond,37
BsmtExposure,38
BsmtFinType1,37
BsmtFinType2,38
Electrical,1
GarageType,81


In [97]:
dfcorr = train1.corr()
numeric = dfcorr.loc[dfcorr['SalePrice']>=0.5].index

OverallQual
YearBuilt
YearRemodAdd
TotalBsmtSF
1stFlrSF
GrLivArea
FullBath
TotRmsAbvGrd
GarageCars
GarageArea
SalePrice


In [113]:
train2=train1.copy()
def scaler(train2):
  for col in numeric:
    try:
      if col is not 'SalePrice':
        train2[col] = (train2[col] - train2[col].min())/(train2[col].max()-train2[col].min())
    except:
      pass
    
scaler(train2)

* train1 itu yang udah dibersihin NA-nya
* train2 itu yang numericnya diambil yg korelasinya tinggi trs di minmaxscaler

##Pycaret

In [105]:
categ = ['MSSubClass','MSZoning','Street','LotShape','LandContour','Utilities','LotConfig','LandSlope','Neighborhood','Condition1','Condition2','BldgType','HouseStyle','RoofStyle','RoofMatl','Exterior1st','Exterior2nd','MasVnrType','ExterQual','ExterCond','Foundation', 'BsmtQual',
'BsmtCond','BsmtExposure', 'BsmtFinType1', 'BsmtFinType2','Heating','HeatingQC','CentralAir','Electrical','KitchenQual','Functional','GarageType','GarageFinish','GarageQual','GarageCond','PavedDrive','MoSold','SaleType','SaleCondition']

In [106]:
initial = pycaret.regression.setup(train1,target='SalePrice',categorical_features=categ)

,Description,Value
0,session_id,3620
1,Target,SalePrice
2,Original Data,"(1460, 75)"
3,Missing Values,True
4,Numeric Features,22
5,Categorical Features,52
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(1021, 362)"


In [107]:
modelkita = pycaret.regression.compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.0229,0.0014,0.0374,0.8655,0.0286,0.2199,0.215
br,Bayesian Ridge,0.0234,0.0014,0.0367,0.8646,0.0281,0.1465,0.155
ridge,Ridge Regression,0.0237,0.0015,0.0369,0.8636,0.0284,0.1668,0.034
gbr,Gradient Boosting Regressor,0.0228,0.0015,0.0379,0.8568,0.0286,0.1668,0.619
rf,Random Forest Regressor,0.0242,0.0016,0.0396,0.8516,0.0302,0.2285,2.074
omp,Orthogonal Matching Pursuit,0.0239,0.0017,0.0388,0.8478,0.0299,0.1518,0.035
et,Extra Trees Regressor,0.0252,0.0017,0.0402,0.8463,0.0308,0.2477,2.205
lr,Linear Regression,0.0261,0.0018,0.0413,0.8307,0.0322,0.2181,0.294
ada,AdaBoost Regressor,0.0319,0.0021,0.0459,0.7996,0.0361,0.3794,0.404
dt,Decision Tree Regressor,0.0377,0.0033,0.0566,0.6909,0.0441,0.3270,0.059


In [89]:
test.drop(['Id','PoolQC','Fence','MiscFeature','FireplaceQu','Alley'],inplace=True,axis=1)

In [115]:
test

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,20,RH,80.0,11622,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,0.444444,6,0.625954,0.183333,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,0.173111,GasA,TA,Y,SBrkr,0.104309,0,0,0.104309,0.0,0.0,0.25,0,2,1,TA,0.166667,Typ,0,Attchd,1961.0,Unf,0.2,0.490591,TA,TA,Y,140,0,0,0,120,0,0,6,2010,WD,Normal
1,20,RL,81.0,14267,Pave,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,0.555556,6,0.603053,0.133333,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,0.260844,GasA,TA,Y,SBrkr,0.196672,0,0,0.196672,0.0,0.0,0.25,1,3,1,Gd,0.250000,Typ,0,Attchd,1958.0,Unf,0.2,0.209677,TA,TA,Y,393,36,0,0,0,0,12500,6,2010,WD,Normal
2,60,RL,74.0,13830,Pave,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,0.444444,5,0.900763,0.800000,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,0.182139,GasA,Gd,Y,SBrkr,0.111135,701,0,0.260666,0.0,0.0,0.50,1,3,1,TA,0.250000,Typ,1,Attchd,1997.0,Fin,0.4,0.323925,TA,TA,Y,212,34,0,0,0,0,0,3,2010,WD,Normal
3,60,RL,78.0,9978,Pave,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,0.555556,6,0.908397,0.800000,Gable,CompShg,VinylSd,VinylSd,BrkFace,20.0,TA,TA,PConc,TA,TA,No,GLQ,602.0,Unf,0.0,324.0,0.181747,GasA,Ex,Y,SBrkr,0.110708,678,0,0.255333,0.0,0.0,0.50,1,3,1,Gd,0.333333,Typ,1,Attchd,1998.0,Fin,0.4,0.315860,TA,TA,Y,360,36,0,0,0,0,0,6,2010,WD,Normal
4,120,RL,43.0,5005,Pave,IR1,HLS,AllPub,Inside,Gtl,StoneBr,Norm,Norm,TwnhsE,1Story,0.777778,5,0.862595,0.700000,Gable,CompShg,HdBoard,HdBoard,None,0.0,Gd,TA,PConc,Gd,TA,No,ALQ,263.0,Unf,0.0,1017.0,0.251227,GasA,Ex,Y,SBrkr,0.186220,0,0,0.186220,0.0,0.0,0.50,0,2,1,Gd,0.166667,Typ,0,Attchd,1992.0,RFn,0.4,0.340054,TA,TA,Y,0,82,0,0,144,0,0,1,2010,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,160,RM,21.0,1936,Pave,Reg,Lvl,AllPub,Inside,Gtl,MeadowV,Norm,Norm,Twnhs,2Story,0.333333,7,0.694656,0.333333,Gable,CompShg,CemntBd,CmentBd,None,0.0,TA,TA,CBlock,TA,TA,No,Unf,0.0,Unf,0.0,546.0,0.107164,GasA,Gd,Y,SBrkr,0.029650,546,0,0.146118,0.0,0.0,0.25,1,3,1,TA,0.166667,Typ,0,NaN,NaN,NaN,0.0,0.000000,NaN,NaN,Y,0,0,0,0,0,0,0,6,2006,WD,Normal
1455,160,RM,21.0,1894,Pave,Reg,Lvl,AllPub,Inside,Gtl,MeadowV,Norm,Norm,TwnhsE,2Story,0.333333,5,0.694656,0.333333,Gable,CompShg,CemntBd,CmentBd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,252.0,Unf,0.0,294.0,0.107164,GasA,TA,Y,SBrkr,0.029650,546,0,0.146118,0.0,0.0,0.25,1,3,1,TA,0.250000,Typ,0,CarPort,1970.0,Unf,0.2,0.192204,TA,TA,Y,0,24,0,0,0,0,0,4,2006,WD,Abnorml
1456,20,RL,160.0,20000,Pave,Reg,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,1Story,0.444444,7,0.618321,0.766667,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,ALQ,1224.0,Unf,0.0,0.0,0.240236,GasA,Ex,Y,SBrkr,0.174275,0,0,0.174275,1.0,0.0,0.25,0,4,1,TA,0.333333,Typ,1,Detchd,1960.0,Unf,0.4,0.387097,TA,TA,Y,474,0,0,0,0,0,0,9,2006,WD,Abnorml
1457,85,RL,62.0,10441,Pave,Reg,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,

In [116]:
final = pycaret.regression.predict_model(modelkita , test)

In [117]:
final

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,Label
0,20,RH,80.0,11622,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,0.444444,6,0.625954,0.183333,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,0.173111,GasA,TA,Y,SBrkr,0.104309,0,0,0.104309,0.0,0.0,0.25,0,2,1,TA,0.166667,Typ,0,Attchd,1961.0,Unf,0.2,0.490591,TA,TA,Y,140,0,0,0,120,0,0,6,2010,WD,Normal,0.131368
1,20,RL,81.0,14267,Pave,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,0.555556,6,0.603053,0.133333,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,0.260844,GasA,TA,Y,SBrkr,0.196672,0,0,0.196672,0.0,0.0,0.25,1,3,1,Gd,0.250000,Typ,0,Attchd,1958.0,Unf,0.2,0.209677,TA,TA,Y,393,36,0,0,0,0,12500,6,2010,WD,Normal,0.178278
2,60,RL,74.0,13830,Pave,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,0.444444,5,0.900763,0.800000,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,0.182139,GasA,Gd,Y,SBrkr,0.111135,701,0,0.260666,0.0,0.0,0.50,1,3,1,TA,0.250000,Typ,1,Attchd,1997.0,Fin,0.4,0.323925,TA,TA,Y,212,34,0,0,0,0,0,3,2010,WD,Normal,0.203832
3,60,RL,78.0,9978,Pave,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,0.555556,6,0.908397,0.800000,Gable,CompShg,VinylSd,VinylSd,BrkFace,20.0,TA,TA,PConc,TA,TA,No,GLQ,602.0,Unf,0.0,324.0,0.181747,GasA,Ex,Y,SBrkr,0.110708,678,0,0.255333,0.0,0.0,0.50,1,3,1,Gd,0.333333,Typ,1,Attchd,1998.0,Fin,0.4,0.315860,TA,TA,Y,360,36,0,0,0,0,0,6,2010,WD,Normal,0.238279
4,120,RL,43.0,5005,Pave,IR1,HLS,AllPub,Inside,Gtl,StoneBr,Norm,Norm,TwnhsE,1Story,0.777778,5,0.862595,0.700000,Gable,CompShg,HdBoard,HdBoard,None,0.0,Gd,TA,PConc,Gd,TA,No,ALQ,263.0,Unf,0.0,1017.0,0.251227,GasA,Ex,Y,SBrkr,0.186220,0,0,0.186220,0.0,0.0,0.50,0,2,1,Gd,0.166667,Typ,0,Attchd,1992.0,RFn,0.4,0.340054,TA,TA,Y,0,82,0,0,144,0,0,1,2010,WD,Normal,0.245883
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,160,RM,21.0,1936,Pave,Reg,Lvl,AllPub,Inside,Gtl,MeadowV,Norm,Norm,Twnhs,2Story,0.333333,7,0.694656,0.333333,Gable,CompShg,CemntBd,CmentBd,None,0.0,TA,TA,CBlock,TA,TA,No,Unf,0.0,Unf,0.0,546.0,0.107164,GasA,Gd,Y,SBrkr,0.029650,546,0,0.146118,0.0,0.0,0.25,1,3,1,TA,0.166667,Typ,0,NaN,NaN,NaN,0.0,0.000000,NaN,NaN,Y,0,0,0,0,0,0,0,6,2006,WD,Normal,0.067404
1455,160,RM,21.0,1894,Pave,Reg,Lvl,AllPub,Inside,Gtl,MeadowV,Norm,Norm,TwnhsE,2Story,0.333333,5,0.694656,0.333333,Gable,CompShg,CemntBd,CmentBd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,252.0,Unf,0.0,294.0,0.107164,GasA,TA,Y,SBrkr,0.029650,546,0,0.146118,0.0,0.0,0.25,1,3,1,TA,0.250000,Typ,0,CarPort,1970.0,Unf,0.2,0.192204,TA,TA,Y,0,24,0,0,0,0,0,4,2006,WD,Abnorml,0.061330
1456,20,RL,160.0,20000,Pave,Reg,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,1Story,0.444444,7,0.618321,0.766667,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,ALQ,1224.0,Unf,0.0,0.0,0.240236,GasA,Ex,Y,SBrkr,0.174275,0,0,0.174275,1.0,0.0,0.25,0,4,1,TA,0.333333,Typ,1,Detchd,1960.0,Unf,0.4,0.387097,TA,TA,Y,474,0,0,0,0,0,0,9,2006,WD,Abnorm

In [118]:
submission = pd.DataFrame({
    'Id':testid,
    'SalePrice':final['Label']
})

submission.to_csv('submission.csv',index=False)
!kaggle competitions submit -c house-prices-advanced-regression-techniques -f submission.csv -m "Using pycaret without any cleaning just using setup"

100% 35.1k/35.1k [00:04<00:00, 8.93kB/s]
Successfully submitted to House Prices - Advanced Regression Techniques